# QIIME 2 classo with Atacama soil microbiome

**environment:** qiime2-2020.2

Use this version of [q2-classo](https://github.com/Leo-Simpson/q2-classo/tree/edbf9d20fed931cbc1cff77f5634bb0dcd783c8a) 

In [ ]:
pip install --upgrade c-lasso

In [ ]:
pip install c-lasso

In [ ]:
pip install zarr

In [ ]:
pip install plotly

In [ ]:
cd ..

In [ ]:
!python setup.py install

In [ ]:
!pip install -e .

In [ ]:
!qiime dev refresh-cache

In [9]:
cd example/data/atacama/

/data/example/data/atacama


## Filter features

In [17]:
# !qiime feature-table filter-features \
#   --i-table atacama-table.qza \
#   --p-min-samples 53 \
#   --o-filtered-table filtered-table.qza

Saved FeatureTable[Frequency] to: filtered-table.qza


## log-contrast and taxa processing

Either collapse at genus level, which is the 'easy way', but not really what we want

In [14]:
!qiime taxa collapse \
  --i-table atacama-table.qza \
  --i-taxonomy classification.qza \
  --p-level 6 \
  --o-collapsed-table genus_table.qza

Saved FeatureTable[Frequency] to: genus_table.qza


In [15]:
!qiime classo transform-features \
     --p-transformation clr \
     --p-coef 0.5 \
     --i-features genus_table.qza \
     --o-x genus_table_clr

Saved FeatureTable[Design] to: genus_table_clr.qza


In [16]:
!qiime feature-table summarize \
    --i-table filtered-table.qza \
    --o-visualization table-summary.qzv

Saved Visualization to: table-summary.qzv


In [17]:
!qiime classo transform-features \
     --p-transformation clr \
     --p-coef 0.5 \
     --i-features atacama-table.qza \
     --o-x xclr

Saved FeatureTable[Design] to: xclr.qza


In [21]:
!qiime classo add-taxa \
	--i-features xclr.qza  \
	--i-taxa classification.qza \
	--o-x xtaxa \
    --o-aweights wtaxa

Saved FeatureTable[Design] to: xtaxa.qza
Saved Weights to: wtaxa.qza


## Add covariates

In [22]:
!qiime classo add-covariates \
    --i-features xtaxa.qza \
    --i-weights wtaxa.qza \
    --m-covariates-file atacama-sample-metadata.tsv \
    --p-to-add elevation ph	toc	ec average-soil-relative-humidity average-soil-temperature \
    --p-w-to-add 1. 0.1 0.1 0.1 1. 1. \
    --o-new-features xcovariates \
    --o-new-c ccovariates \
    --o-new-w wcovariates

Saved FeatureTable[Design] to: xcovariates.qza
Saved ConstraintMatrix to: ccovariates.qza
Saved Weights to: wcovariates.qza


## Split table

Split data into training and testing sets : 

In [25]:
!qiime sample-classifier split-table \
	--i-table xcovariates.qza \
	--m-metadata-file atacama-sample-metadata.tsv \
	--m-metadata-column average-soil-temperature \
	--p-test-size 0.2 \
	--p-random-state 42 \
	--p-stratify False \
	--o-training-table regress-xtraining \
	--o-test-table regress-xtest

### in newer version of QIIME use these parameters
    # --o-training-targets training-targets.qza \
    # --o-test-targets test-targets.qza

Saved FeatureTable[Design] to: regress-xtraining.qza
Saved FeatureTable[Design] to: regress-xtest.qza


In [26]:
!qiime sample-classifier split-table \
	--i-table xcovariates.qza \
	--m-metadata-file atacama-sample-metadata.tsv \
	--m-metadata-column vegetation  \
	--p-test-size 0.2 \
	--p-random-state 42 \
	--p-stratify False \
	--o-training-table classify-xtraining \
	--o-test-table classify-xtest

### in newer version of QIIME use these parameters
    # --o-training-targets training-targets.qza \
    # --o-test-targets test-targets.qza

Saved FeatureTable[Design] to: classify-xtraining.qza
Saved FeatureTable[Design] to: classify-xtest.qza


## Regression task 

Apply classo to the training set to solve the linear regression problem : 

In [27]:
!qiime classo regress  \
    --i-features regress-xtraining.qza \
    --i-c ccovariates.qza \
    --i-weights wcovariates.qza \
    --m-y-file atacama-sample-metadata.tsv \
    --m-y-column average-soil-temperature  \
    --p-concomitant \
    --p-stabsel \
    --p-cv \
    --p-path \
    --p-lamfixed \
    --p-stabsel-threshold 0.5 \
    --p-cv-seed 1 \
    --p-no-cv-one-se \
    --o-result regresstaxa

Saved CLASSOProblem to: regresstaxa.qza


## Classification task

In [28]:
!qiime classo classify  \
    --i-features classify-xtraining.qza \
    --i-c ccovariates.qza \
    --i-weights wcovariates.qza \
    --m-y-file atacama-sample-metadata.tsv \
    --m-y-column vegetation  \
    --p-huber \
    --p-stabsel \
    --p-cv \
    --p-path \
    --p-lamfixed \
    --p-stabsel-threshold 0.5 \
    --p-cv-seed 42 \
    --p-no-cv-one-se \
    --o-result classifytaxa

Saved CLASSOProblem to: classifytaxa.qza


## Prediction 

In [29]:
!qiime classo predict \
    --i-features regress-xtest.qza \
    --i-problem regresstaxa.qza \
    --o-predictions regress-predictions.qza

Saved CLASSOProblem to: regress-predictions.qza


In [30]:
!qiime classo predict \
    --i-features classify-xtest.qza \
    --i-problem classifytaxa.qza \
    --o-predictions classify-predictions.qza

Saved CLASSOProblem to: classify-predictions.qza


## Visualization

In [43]:
!qiime classo summarize \
  --i-problem regresstaxa.qza \
  --i-taxa classification.qza \
  --i-predictions regress-predictions.qza \
  --o-visualization regresstaxa_R3.qzv

Saved Visualization to: regresstaxa_R3.qzv


In [44]:
!qiime classo summarize \
  --i-problem classifytaxa.qza \
  --i-taxa classification.qza \
  --i-predictions classify-predictions.qza \
  --o-visualization classifytaxa_C2.qzv

Saved Visualization to: classifytaxa_C2.qzv


Drag&drop .qzv files on : https://view.qiime2.org
Thanks to this alternative, one can also track the workflow that the qiime2 artifact did. 